# Daily Tweet Collection
Iterate all existing user accounts and get all their tweets since `since_id` (last collected tweet, ideally the day before) or `misc.CONIFG["oldest_tweet"]`

Additionally:
* identify suspensions on a daily basis and mark the suspension day.

In [ ]:
# required imports to access api_db, misc, misc.CONFIG, ...
import sys
sys.path = ['.', '..', '../..'] + sys.path
from collection import *

### Conditional Execution
Each file needs to verify if it should be executed or not based on the configurations (for some files this is not optional but all should have this section, even if it is tautological). Example:
```python
if not misc.CONFIG["collection"]["execute_this_script"]: exit()
```

In [ ]:
# Conditional execution
pass

<hr>
<h1 align="center">driver code</h1>

The users where we will search for tweets are users that either don't have a `most_common_language` yet or whose `most_common_language` is in `config.collection.search_languages` and with `depth<=2`

In [ ]:
def task(skip, limit):
    from collections import Counter

    oldest_t = misc.CONFIG["collection"]["oldest_tweet"]
    since_id_key = "since_id"
    print("Collection with oldest tweet at %s and key for since_id '%s'" % (oldest_t, since_id_key))


    search_languages = misc.CONFIG["collection"]["search_languages"]
    find_params = find_exclude_invalid({
        "depth": {"$lte": 1}
    })
    users = api_db.col_users.find(find_params, {since_id_key: True}, no_cursor_timeout=True).skip(skip).limit(limit)

    for u in users:
        print("getting tweets for: %s..." % u["_id"], end="", flush=True)
        tweets = get_tweets(u, api_db.api.GetUserTimeline, since_id_key, oldest_t, {"trim_user":True})
        insert_tweets(tweets)
        # update_most_common_language(u, tweets) # removed because not useful here
        print("got %d new tweets, done." % len(tweets))

In [ ]:
search_languages = misc.CONFIG["collection"]["search_languages"]
find_params = find_exclude_invalid({
    "depth": {"$lte": 1}
})
total = api_db.col_users.count(find_params)
print("Total to process: %d" % total)

In [ ]:
dp = DynamicParallelism(total, task, "tweet_collection", batch_size=50_000, max_threads=misc.CONFIG["collection"]["max_threads"])

In [ ]:
dp.run()

In [ ]:
print("DONE")